In [22]:
import pandas as pd

# 1. 迁移流
mig = pd.read_csv('../data/processed/migration_flow_cleaned.csv')
print("迁移流表 columns:", mig.columns.tolist())
print(mig.head())

# 2. GDP
gdp = pd.read_csv('../data/processed/gdp_country_year_2019_2022_cleaned.csv')
print("GDP表 columns:", gdp.columns.tolist())
print(gdp.head())

# 3. Market Access
ma = pd.read_csv('../data/processed/market_access_panel.csv')
print("Market Access表 columns:", ma.columns.tolist())
print(ma.head())

# 4. SPEI
spei = pd.read_csv('../data/processed/spei03_country_month_cleaned.csv')
print("SPEI表 columns:", spei.columns.tolist())
print(spei.head())

# 5. 边境摩擦
border = pd.read_csv('../data/processed/border_friction_panel_ew.csv')
print("边境摩擦表 columns:", border.columns.tolist())
print(border.head())

迁移流表 columns: ['origin_iso3', 'destination_iso3', 'migration_month', 'year', 'month', 'flow', 'log_flow', 'origin_iso2', 'destination_iso2']
  origin_iso3 destination_iso3 migration_month  year  month  flow  log_flow  \
0         AND              ARE      2019-01-01  2019      1    12  2.564949   
1         AND              ARE      2019-02-01  2019      2     2  1.098612   
2         AND              ARE      2019-03-01  2019      3     1  0.693147   
3         AND              ARE      2019-04-01  2019      4     7  2.079442   
4         AND              ARE      2019-05-01  2019      5     0  0.000000   

  origin_iso2 destination_iso2  
0          AD               AE  
1          AD               AE  
2          AD               AE  
3          AD               AE  
4          AD               AE  
GDP表 columns: ['iso3', 'year', 'gdp']
  iso3  year           gdp
0  AFE  2019  1.009747e+12
1  AFW  2019  8.332889e+11
2  ARB  2019  2.949355e+12
3  CSS  2019  6.192930e+10
4  CEB  2019 

In [23]:
# ...existing code...

# 1. 检查主键/连接字段
print(mig.columns)
print(gdp.columns)
print(ma.columns)
print(spei.columns)
print(border.columns)

# 2. 检查缺失值
print(mig.isnull().sum())
print(gdp.isnull().sum())
print(ma.isnull().sum())
print(spei.isnull().sum())
print(border.isnull().sum())

# 3. 检查数据类型
print(mig.dtypes)
print(gdp.dtypes)
print(ma.dtypes)
print(spei.dtypes)
print(border.dtypes)

# 4. 如有需要，统一字段名和类型
# 例如：mig['country_code'] = mig['country_code'].astype(str)

# 5. 去除重复值
print(mig.duplicated().sum())
print(gdp.duplicated().sum())
print(ma.duplicated().sum())
print(spei.duplicated().sum())
print(border.duplicated().sum())
# ...existing code...

Index(['origin_iso3', 'destination_iso3', 'migration_month', 'year', 'month',
       'flow', 'log_flow', 'origin_iso2', 'destination_iso2'],
      dtype='object')
Index(['iso3', 'year', 'gdp'], dtype='object')
Index(['iso3', 'year', 'MA'], dtype='object')
Index(['country', 'ISO_A3', 'date', 'spei'], dtype='object')
Index(['iso_o', 'iso_d', 'contig', 'comlang_off', 'colony', 'border_friction'], dtype='object')
origin_iso3         0
destination_iso3    0
migration_month     0
year                0
month               0
flow                0
log_flow            0
origin_iso2         0
destination_iso2    0
dtype: int64
iso3     0
year     0
gdp     36
dtype: int64
iso3    0
year    0
MA      0
dtype: int64
country    0
ISO_A3     0
date       0
spei       0
dtype: int64
iso_o              0
iso_d              0
contig             0
comlang_off        0
colony             0
border_friction    0
dtype: int64
origin_iso3          object
destination_iso3     object
migration_month      object

In [24]:
# ...existing code...

# 统一字段名
gdp = gdp.rename(columns={'iso3': 'iso'})
ma = ma.rename(columns={'iso3': 'iso'})
spei = spei.rename(columns={'ISO_A3': 'iso'})
border = border.rename(columns={'iso_o': 'iso_o', 'iso_d': 'iso_d'})  # 已一致

# 处理SPEI的时间字段
spei[['year', 'month']] = spei['date'].str.split('-', expand=True)[[0,1]].astype(int)

# 检查处理结果
print(spei[['iso', 'year', 'month']].head())

# 现在可以用 iso_o/iso_d, year, month 作为主键进行merge

   iso  year  month
0  AFG  2019      1
1  ALA  2019      1
2  ALB  2019      1
3  DZA  2019      1
4  AGO  2019      1


In [ ]:
# 5. 合并origin地的SPEI
mig = mig.merge(spei, left_on=['origin_iso3', 'year', 'month'], right_on=['iso', 'year', 'month'], how='left')
mig = mig.rename(columns={'spei': 'spei_o','country':'country_o'}).drop(columns=['iso','date'])

display(mig.head())


,origin_iso3,destination_iso3,migration_month,year,month,flow,log_flow,origin_iso2,destination_iso2,country_o,date,spei_o
0,AND,ARE,2019-01-01,2019,1,12,2.564949,AD,AE,NaN,NaN,NaN
1,AND,ARE,2019-02-01,2019,2,2,1.098612,AD,AE,NaN,NaN,NaN
2,AND,ARE,2019-03-01,2019,3,1,0.693147,AD,AE,NaN,NaN,NaN
3,AND,ARE,2019-04-01,2019,4,7,2.079442,AD,AE,NaN,NaN,NaN
4,AND,ARE,2019-05-01,2019,5,0,0.000000,AD,AE,NaN,NaN,NaN


In [29]:
# ...existing code...

# 缺失origin地SPEI的总数和占比
total_rows = len(mig)
missing_count = mig['spei_o'].isnull().sum()
missing_ratio = missing_count / total_rows

print(f"缺失origin地SPEI的总数：{missing_count}")
print(f"总行数：{total_rows}")
print(f"缺失占比：{missing_ratio:.2%}")

缺失origin地SPEI的总数：146016
总行数：1545874
缺失占比：9.45%


In [30]:
# ...existing code...

# 查看缺失spei_o的分布（按国家、年份统计）
missing_spei = mig[mig['spei_o'].isnull()]
print("缺失SPEI的origin国家分布：")
print(missing_spei['origin_iso3'].value_counts())
print("缺失SPEI的年份分布：")
print(missing_spei['year'].value_counts())

# 合并后spei_o的基本统计
print("合并后spei_o的描述性统计：")
print(mig['spei_o'].describe())

缺失SPEI的origin国家分布：
origin_iso3
AND    8592
BRB    8592
BHR    8592
FSM    8592
FRA    8592
GRD    8592
HKG    8592
KIR    8592
LCA    8592
MAC    8592
MLT    8592
MDV    8592
NOR    8592
SGP    8592
STP    8592
TON    8592
XKX    8544
Name: count, dtype: int64
缺失SPEI的年份分布：
year
2019    36504
2020    36504
2021    36504
2022    36504
Name: count, dtype: int64
合并后spei_o的描述性统计：
count    1.399858e+06
mean    -2.661201e-01
std      8.681227e-01
min     -3.313311e+00
25%     -8.599656e-01
50%     -2.236114e-01
75%      2.752515e-01
max      2.604590e+00
Name: spei_o, dtype: float64
